In [0]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

In [0]:
test = pd.read_csv('profile_data_test.csv')
train = pd.read_csv('profile_data_train.csv')

all_data = pd.concat((test.loc[:,'handle':'avg_word_count'], train.loc[:,'handle':'avg_word_count'])).reset_index(drop=True)

In [0]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]

y_train = train['age_group']
y_test = test['age_group']

In [4]:
# first try xgboost on numerical features only
import copy

X_train_num = copy.deepcopy(X_train)
del X_train_num['handle']
del X_train_num['tweets_text']
del X_train_num['hashtags']

n_features = X_train_num.shape[1]

X_train_num.head()

,followers,following,follower_following_ratio,avg_favorites,min_favorites,max_favorites,avg_retweets,min_retweets,max_retweets,hashtags_per_tweet,mentions_per_tweet,avg_timedelta_hrs,avg_word_count
0,126790.0,38762.0,3.270987,367.000,207.0,663.0,31.800,18.0,64.0,0.50,0.1,26.393673,166
1,1272.0,426.0,2.985915,10.700,0.0,55.0,6.300,0.0,52.0,0.20,0.2,217.188333,81
2,74947.0,49.0,1529.530612,181.100,42.0,774.0,15.400,3.0,53.0,0.50,0.1,88.825556,386
3,560650.0,119.0,4711.344538,23.400,4.0,94.0,1.400,0.0,6.0,0.10,0.0,1.508302,104
4,392302.0,3048.0,128.708005,210.125,3.0,514.0,47.125,2.0,141.0,0.75,0.5,20.227262,281


In [5]:
X_test_num = copy.deepcopy(X_test)
del X_test_num['handle']
del X_test_num['tweets_text']
del X_test_num['hashtags']

X_test_num.head()

,followers,following,follower_following_ratio,avg_favorites,min_favorites,max_favorites,avg_retweets,min_retweets,max_retweets,hashtags_per_tweet,mentions_per_tweet,avg_timedelta_hrs,avg_word_count
1699,3946343.0,69.0,57193.376812,5167.4,131.0,24727.0,163.5,1.0,729.0,0.7,0.1,8.068981,109
1700,54359.0,17.0,3197.588235,548.5,234.0,1230.0,58.7,17.0,110.0,0.0,0.0,8.208519,54
1701,778.0,659.0,1.180577,2.6,0.0,13.0,0.3,0.0,2.0,0.0,0.0,20.997809,121
1702,114.0,148.0,0.770270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.273642,54
1703,256.0,207.0,1.236715,4.4,0.0,28.0,1.2,0.0,5.0,0.0,0.1,490.341358,76


In [6]:
!pip install scikit-optimize

     |████████████████████████████████| 81kB 2.2MB/s 


In [7]:
all_metadata = pd.concat((X_train_num.loc[:,'followers':'avg_word_count'], X_test_num.loc[:,'followers':'avg_word_count'])).reset_index(drop=True)
y = pd.concat((y_train, y_test))

all_metadata.head()

,followers,following,follower_following_ratio,avg_favorites,min_favorites,max_favorites,avg_retweets,min_retweets,max_retweets,hashtags_per_tweet,mentions_per_tweet,avg_timedelta_hrs,avg_word_count
0,126790.0,38762.0,3.270987,367.000,207.0,663.0,31.800,18.0,64.0,0.50,0.1,26.393673,166
1,1272.0,426.0,2.985915,10.700,0.0,55.0,6.300,0.0,52.0,0.20,0.2,217.188333,81
2,74947.0,49.0,1529.530612,181.100,42.0,774.0,15.400,3.0,53.0,0.50,0.1,88.825556,386
3,560650.0,119.0,4711.344538,23.400,4.0,94.0,1.400,0.0,6.0,0.10,0.0,1.508302,104
4,392302.0,3048.0,128.708005,210.125,3.0,514.0,47.125,2.0,141.0,0.75,0.5,20.227262,281


In [37]:
all_metadata = all_metadata.replace([np.inf, -np.inf], np.nan)
all_metadata = all_metadata.fillna(0)
y.fillna(0)

all_metadata.head()

,followers,following,follower_following_ratio,avg_favorites,min_favorites,max_favorites,avg_retweets,min_retweets,max_retweets,hashtags_per_tweet,mentions_per_tweet,avg_timedelta_hrs,avg_word_count
0,126790.0,38762.0,3.270987,367.000,207.0,663.0,31.800,18.0,64.0,0.50,0.1,26.393673,166
1,1272.0,426.0,2.985915,10.700,0.0,55.0,6.300,0.0,52.0,0.20,0.2,217.188333,81
2,74947.0,49.0,1529.530612,181.100,42.0,774.0,15.400,3.0,53.0,0.50,0.1,88.825556,386
3,560650.0,119.0,4711.344538,23.400,4.0,94.0,1.400,0.0,6.0,0.10,0.0,1.508302,104
4,392302.0,3048.0,128.708005,210.125,3.0,514.0,47.125,2.0,141.0,0.75,0.5,20.227262,281


In [49]:
np.all(np.isfinite(all_metadata))

True

In [0]:
# gradient boosted trees tend to do well on problems like this
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=500)

In [0]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args

# The list of hyper-parameters we want to optimize. For each one we define the
# bounds, the corresponding scikit-learn parameter name, as well as how to
# sample values from that dimension (`'log-uniform'` for the learning rate)
space  = [Integer(3, 20, name='max_depth'),
          Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
          Integer(1, 100, name='min_child_weight'),
          Integer(100, 1000, name='n_estimators'),
          Integer(0, 10, name='gamma')]

# this decorator allows your objective function to receive a the parameters as
# keyword arguments. This is particularly convenient when you want to set
# scikit-learn estimator parameters
@use_named_args(space)
def objective(**params):
    xgb.set_params(**params)

    return -np.mean(cross_val_score(xgb, X=all_metadata, y=y, cv=5, n_jobs=-1,
                                    scoring="accuracy"))


In [92]:
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=50, random_state=0, verbose=True)

"Best score=%.4f" % res_gp.fun

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 19.3584
Function value obtained: -0.2971
Current minimum: -0.2971
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 22.6152
Function value obtained: -0.3023
Current minimum: -0.3023
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 32.5237
Function value obtained: -0.2990
Current minimum: -0.3023
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 4.5399
Function value obtained: -0.2980
Current minimum: -0.3023
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 21.5602
Function value obtained: -0.2980
Current minimum: -0.3023
Iteration No: 6 start

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Iteration No: 17 ended. Search finished for the next optimal point.
Time taken: 139.7383
Function value obtained: -0.2980
Current minimum: -0.3074
Iteration No: 18 started. Searching for the next optimal point.
Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 3.9155
Function value obtained: -0.2980
Current minimum: -0.3074
Iteration No: 19 started. Searching for the next optimal point.
Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 21.1246
Function value obtained: -0.2980
Current minimum: -0.3074
Iteration No: 20 started. Searching for the next optimal point.
Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 2.3906
Function value obtained: -0.2980
Current minimum: -0.3074
Iteration No: 21 started. Searching for the next optimal point.
Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 2.3796
Function value obtained: -0.2980
Current minimum: -0.3074
Iteration No: 22

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 35 ended. Search finished for the next optimal point.
Time taken: 2.5949
Function value obtained: -0.2980
Current minimum: -0.3074
Iteration No: 36 started. Searching for the next optimal point.
Iteration No: 36 ended. Search finished for the next optimal point.
Time taken: 4.2218
Function value obtained: -0.2980
Current minimum: -0.3074
Iteration No: 37 started. Searching for the next optimal point.
Iteration No: 37 ended. Search finished for the next optimal point.
Time taken: 23.0872
Function value obtained: -0.2980
Current minimum: -0.3074
Iteration No: 38 started. Searching for the next optimal point.
Iteration No: 38 ended. Search finished for the next optimal point.
Time taken: 109.9419
Function value obtained: -0.2980
Current minimum: -0.3074
Iteration No: 39 started. Searching for the next optimal point.
Iteration No: 39 ended. Search finished for the next optimal point.
Time taken: 3.1533
Function value obtained: -0.2933
Current minimum: -0.3074
Iteration No: 40

...

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 20.5774
Function value obtained: -0.2980
Current minimum: -0.3074
Iteration No: 46 started. Searching for the next optimal point.
Iteration No: 46 ended. Search finished for the next optimal point.
Time taken: 15.1837
Function value obtained: -0.3009
Current minimum: -0.3074
Iteration No: 47 started. Searching for the next optimal point.
Iteration No: 47 ended. Search finished for the next optimal point.
Time taken: 32.7993
Function value obtained: -0.2976
Current minimum: -0.3074
Iteration No: 48 started. Searching for the next optimal point.
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 2.6197
Function value obtained: -0.2961
Current minimum: -0.3074
Iteration No: 49 started. Searching for the next optimal point.
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 19.6199
Function value obtained: -0.3004
Current minimum: -0.3074
Iteration No: 5

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 142.4908
Function value obtained: -0.2980
Current minimum: -0.3074


'Best score=-0.3074'

In [93]:
print("""Best parameters:
- max_depth=%d
- learning_rate=%.6f
- min_child_weight=%d
- n_estimators=%d
- gamma=%d""" % (res_gp.x[0], res_gp.x[1],
                            res_gp.x[2], res_gp.x[3],
                            res_gp.x[4]))

Best parameters:
- max_depth=6
- learning_rate=0.048372
- min_child_weight=22
- n_estimators=222
- gamma=3


In [94]:
xgb = XGBClassifier(max_depth=6, learning_rate=0.048372, min_child_weight=22, n_estimators=222, gamma=3)
xgb.fit(X_train_num, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=3,
              learning_rate=0.048372, max_delta_step=0, max_depth=6,
              min_child_weight=22, missing=None, n_estimators=222, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [95]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, xgb.predict(X_test_num))

0.31529411764705884